In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
### Distribution ✨ ❇ Demo 💪 ### 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

### fancy logging variables, philosophically reminding us of the goal ###
fancy = dict(
		pe		= r'$V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'$E',						
		log_psi	= r'$\log\psi', 			
		deltar	= r'$\delta_\mathrm{r}',	
		x		= r'$r_\mathrm{e}',
)

### pyfig ###
from pyfig import Pyfig

# arg = {
# 	'a_z':[4,], 
# 	'n_b': 256, 
# 	'n_sv': 16, 
# 	'n_pv': 16, 
# 	'n_corr': 20, 
# 	'n_step': 100000, 
# 	'log_metric_step': 1, 
# 	'exp_name':'demo',
# }


c = Pyfig(wb_mode='online', submit=False, run_sweep=False, notebook=True)

n_device = c.n_device
print(f'🤖 {n_device} GPUs available')


# from pprint import pprint
# pprint(c.d)

""" live plotting in another notebook """
""" copy lines and run in analysis while the exp is live """
# api = wandb.Api()
# run = api.run("<run-here>")
# c = run.config
# h = run.history()
# s = run.summary


# SOLVE THE CONUNDRUM 

init sub classes
updating configuration
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 1e03f0c stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
running script
setting exp_path
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 1e03f0c stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xmax1 (hwat). Use `wandb login --relogin` to force relogin


🤖 1 GPUs available


' copy lines and run in analysis while the exp is live '

In [3]:
import torch
torch.manual_seed(1234)
torch.set_default_tensor_type(torch.DoubleTensor)   # ❗ Ensure works when default not set AND can go float32 or 64
cuda = torch.cuda.is_available()
n_device = torch.cuda.device_count()
current_device = torch.cuda.current_device()
device = torch.cuda.device(0)
device_name = torch.cuda.get_device_name(0)
print('cuda: ', cuda, 'n_dev: ', n_device, 'device', device, 'name: ', device_name)

cuda:  True n_dev:  1 device <torch.cuda.device object at 0x7f6974e8f5b0> name:  NVIDIA TITAN Xp


In [4]:
device = 'cuda'

In [5]:
### model ###
from functools import partial
from hwat import Ansatz_fb
from torch import nn

import pprint

from hwat import init_r, get_center_points
x = torch.randn(1)
c._convert(device, dtype=x.dtype)
n_e = c.data.n_e
center_points = get_center_points(c.data.n_e, c.data.a)
r = init_r(n_device, c.data.n_b, c.data.n_e, center_points, std=0.1).to(device)
dtype = r.dtype

print('r: ', r.shape, r.dtype)
r = r[0] # single batch, single gpu, ❗ how to multi gpu
model = c.partial(Ansatz_fb).to(device)

model_check = c.partial(Ansatz_fb, with_sign=True).to(device)
r_swap = r[0, [1,0]+[i for i in range(2, c.data.n_e)]]
lp0, s0 = model_check(r[0])
lp1, s1 = model_check(r_swap)
print('anti-symmetry: ', lp0.item(), lp0.item(), s0.item(), s1.item())
r_swap = r[0, [i for i in range(0, c.data.n_e-2)]+[n_e-1,n_e-2]]
lp1, s1 = model_check(r_swap)
print('anti-symmetry: ', lp0.item(), lp0.item(), s0.item(), s1.item())

pprint.pprint(c.d)

r:  torch.Size([1, 256, 4, 3]) torch.float64
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 1e03f0c stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 


IndexError: list index out of range

In [ ]:
### train step ###
from hwat import compute_ke_b, compute_pe_b
from functorch import vmap, make_functional, grad

def train_step(model, r):

	model_fn, params = make_functional(model)
	model_fn = vmap(model_fn, in_dims=(None, 0))
	model_rv = lambda _r: model_fn(params, _r)
	model_gv = lambda _params: model_fn(_params, r)

	# with torch.no_grad():
	ke = compute_ke_b(model_rv, r)
	pe = compute_pe_b(r, c.data.a, c.data.a_z)
	# print(pe.shape, ke.shape)
	e = -0.5*ke + pe 
	e_mean_dist = torch.mean(torch.abs(torch.median(e) - e))
	# print(e.shape, e_mean_dist.shape)
	e_clip = torch.clip(e, min=e-5*e_mean_dist, max=e+5*e_mean_dist)

	loss = lambda _params: ((e_clip - e_clip.mean())*model_gv(_params)).mean()
	grad_fn = grad(loss)	
	grads = grad_fn(params)

	for p, g in zip(model.parameters(), grads):    # ❗ do we want functional or other design?! 
		p.grad = g.clone()
  
	# loss.backward()
	grads = [p.detach().cpu().numpy() for p in model.parameters()]
	params = [p.detach().cpu().numpy() for p in model.parameters()]  

	v_tr = dict(
		params=params, grads=grads,
		e=e, pe=pe, ke=ke,
		r=r
	)
	return v_tr

r = r.to(device)
center_points = center_points.to(device)

v_tr = train_step(model, r)
pprint.pprint({k:v.shape if isinstance(v, torch.Tensor) else len(v) for k,v in v_tr.items()})


### init variables ###
deltar = torch.tensor([0.02]).to(device)

print(f"""exp/actual | 
	cps    : {(c.data.n_e,3)}/{center_points.shape}
	r      : {(c.n_device, c.data.n_b, c.data.n_e, 3)}/{r.shape}
	deltar : {(1,)}/{deltar.shape}
""")


### init functions ### 
from hwat import sample_b

### train ###
import wandb
from hwat import keep_around_points
from utils import compute_metrix
from time import time
t0 = time()

### add in optimiser
opt = torch.optim.Adam(model.parameters(), lr=0.001)
c.log_metric_step = 1
### fix sampler
### fix train step 
### metrix conversion
# print(c.wandb_c.)
wandb.define_metric("*", step_metric="tr/step")
print('Go see:', c._run.url)
for step in range(1, c.n_step+1):
	
	model_fn, params = make_functional(model)
	model_fn = vmap(model_fn, in_dims=(None, 0))
	model_rv = lambda _r: model_fn(params, _r)
 
	r, acc, deltar = sample_b(model_rv, r, deltar, n_corr=c.data.n_corr)  # ❗needs testing  ✅
	r = keep_around_points(r, center_points, l=2.) if step < 1000 else r
	
	opt.zero_grad()
	v_tr = train_step(model, r)
	opt.step()
	
	
	if not (step % c.log_metric_step):
		v_tr |= dict(acc=0.0, deltar=deltar)
		metrix = compute_metrix(v_tr)  # ❗ needs converting to torch, ie tree maps ✅
		wandb.log({'tr/step':step, **metrix})
	
	diff = time()-t0
	if diff > 100: 
		t0 = time()
		pprint.pprint({k:f'{v.mean().item():.4f}' for k,v in v_tr.items() if isinstance(v, torch.Tensor)} | {'step': step})
  

{'e': torch.Size([256]),
 'grads': 18,
 'ke': torch.Size([256]),
 'params': 18,
 'pe': torch.Size([256]),
 'r': torch.Size([256, 4, 3])}
exp/actual | 
	cps    : (4, 3)/torch.Size([4, 3])
	r      : (1, 256, 4, 3)/torch.Size([256, 4, 3])
	deltar : (1,)/torch.Size([1])

Go see: https://wandb.ai/hwat/hwat/runs/2qztkgat
{'e': '-8.7969', 'ke': '-7.6951', 'pe': '-12.6444', 'r': '-0.0185'}
{'e': '-8.2510', 'ke': '-8.9282', 'pe': '-12.7151', 'r': '-0.0201'}
{'e': '-13.1621', 'ke': '-5.3839', 'pe': '-15.8541', 'r': '0.0301'}
{'e': '-18.9316', 'ke': '9.9038', 'pe': '-13.9797', 'r': '-0.0116'}
{'e': '-3.5923', 'ke': '-16.1036', 'pe': '-11.6441', 'r': '0.0276'}
{'e': '-6.4631', 'ke': '-9.5289', 'pe': '-11.2276', 'r': '0.0079'}
{'e': '-4.4273', 'ke': '-14.1766', 'pe': '-11.5156', 'r': '0.0239'}
{'e': '-4.8130', 'ke': '-13.6131', 'pe': '-11.6195', 'r': '0.0156'}
{'e': '-3.9907', 'ke': '-15.1778', 'pe': '-11.5796', 'r': '0.0302'}
{'e': '-3.8649', 'ke': '-13.4845', 'pe': '-10.6072', 'r': '-0.0087'}


In [ ]:
# ```{toggle} env vars and jax debug config notes
# ❇️ Magic & debug not currently used

# %load_ext autoreload
# %autoreload 2
# %env CUDA_VISIBLE_DEVICES='3'
# %env "WANDB_NOTEBOOK_NAME" "run.ipynb" # ❕same as notebook

# from jax.config import config
# config.update('jax_disable_jit', True)
# ```